## 1. 다음 영화에서 정보 가져오고 세부정보로 

In [1]:
from bs4 import BeautifulSoup
import urllib.request as REQ

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

from datetime import datetime 

# 폰트 설정
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

# 구성
- 1. 홈에서 겉 내용 크롤링하는 함수 생성
- 2. 각 영화의 세부페이지에서 내용 크롤링하는 함수 생성
- 3. 각 페이지를 넘기면서 이동하여 1,2의 함수를 실행

In [120]:
movie_url = 'https://movie.daum.net/'

# 홈 정보 크롤링
response    = REQ.urlopen( movie_url+'premovie/released' )
soup        = BeautifulSoup( response, 'html5lib' )

paging = soup.select('a.link_page')

for page in paging:
    print(movie_url + page['href'])
#     movie( movie_url+ page['href'] )


https://movie.daum.net/released?reservationOnly=N&page=2
https://movie.daum.net/released?reservationOnly=N&page=3


In [122]:
# (원본) 1.  영화 사이트에서 기본 정보 가져오기


movie_url1 = 'https://movie.daum.net/premovie/released'
response = REQ.urlopen(movie_url1)
soup = BeautifulSoup( response, 'html5lib' )
div = soup.select('div#mArticle')

movie= []

for i in div:
    for k in i.select('li'):
        movie.append( {  '평점'    : k.select_one('.emph_grade').string,
                         '영화제목': k.select_one('.link_g').string,
                         '상세주소': k.select_one('.link_g')['href'],
                         '개봉일'  : pd.to_datetime( k.select_one('dd').string[:-3] )
                     } )
        
movie = pd.DataFrame( movie )

# 2.  각 영화 페이지에 들어가서 상세 정보 가져오기
movie_url2 = 'https://movie.daum.net/'
movie['상세주소']

total_detail = []

for url in movie['상세주소']:
    response = REQ.urlopen( movie_url2 + url )
    soup = BeautifulSoup( response, 'html5lib' )
    
    detail_list = []
    detail_url  = soup.select_one( 'div.detail_movie.detail_main' ) 
    
    # (1) 첫 번째 div 상세 보기
    for i in detail_url.select('.movie_summary > dl > dd'):
        detail_list.append(i.text.strip().replace(' ', '').replace( '\n', '').replace( '감독', '').replace( '주연', ''))
        
    # **'재개봉'이 있는 경우 위해 인덱싱 조정**    
    if len(detail_list) == 6:
        running_time = detail_list[3].split(',')[0]
        appropriate_age  = detail_list[3].split(',')[1]
        movie_director = detail_list[4]
        main_actors = detail_list[5]
    else:
        running_time = detail_list[4].split(',')[0]
        appropriate_age  = detail_list[4].split(',')[1]
        movie_director = detail_list[5]
        main_actors = detail_list[6]
        
    dic = {'장르': detail_list[0] , 
           '국가': detail_list[1], 
           '러닝 타임':running_time, 
           '관람 연령' :appropriate_age,
           '영화 감독': movie_director,
           '주연 배우': main_actors,
           # (2) 두 번째 div 상세보기
           '줄거리' : detail_url.select_one('.desc_movie > p' ).text.replace( '\n', '').replace( '\xa0', '').strip() }
    total_detail.append( dic )
#     print(dic)

total_detail = pd.DataFrame( total_detail )

Movie_Data = pd.concat( [movie, total_detail], axis=1 )
Movie_Data.set_index( '영화제목')

,평점,상세주소,개봉일,장르,국가,러닝 타임,관람 연령,영화 감독,주연 배우,줄거리
영화제목,,,,,,,,,,
원더 우먼 1984,4.9,/moviedb/main?movieId=115600,2020-12-23,액션/어드벤처/판타지,미국,151분,12세이상관람가,패티젠킨스,"갤가돗,크리스파인","세상이 기다린 히어로, 희망이 되다!1984년 모든 것이 활기찬 시대, 다이애나 프..."
가을의 마티네,9.7,/moviedb/main?movieId=136593,2020-12-31,로맨스/멜로/드라마,일본,123분,12세이상관람가,니시타니히로시,"후쿠야마마사하루,이시다유리코","파리, 마드리드, 뉴욕을 오가는 운명적 사랑어디에 있든 내 연주가 당신에게 닿기를천..."
나이팅게일,6.0,/moviedb/main?movieId=123852,2020-12-30,스릴러,오스트레일리아,136분,청소년관람불가,제니퍼켄트,"아이슬링프란시오시,샘클래플린,베이칼리가남바르","그날 밤, 나의 복수는 시작됐다호주 태즈메이니아의 마을.아름다운 노랫소리로 나이팅게..."
화양연화,8.5,/moviedb/main?movieId=2361,2020-12-24,로맨스/멜로/드라마,"프랑스,홍콩",99분,15세이상관람가,왕가위,"장만옥,양조위",화양연화花樣年華가장 아름답고 찬란했던 시절 같은 날 같은 아파트로 이사 온 ‘첸 부...
럭키 몬스터,6.5,/moviedb/main?movieId=134954,2020-12-03,드라마,한국,93분,청소년관람불가,봉준영,"김도윤,장진희,박성준",★빚쟁이에서 로또 1등 당첨까지기적의 ‘도맹수’가 사라진 아내 ‘성리아’를 애타게 ...
조제,6.9,/moviedb/main?movieId=130427,2020-12-10,로맨스/멜로,한국,117분,15세이상관람가,김종관,"한지민,남주혁",자신을 ‘조제’로 불러달라는 그녀처음 만난 그날부터 ‘조제’는 ‘영석’에게 잊을 수...
라라랜드,8.0,/moviedb/main?movieId=95306,2020-12-31,로맨스/멜로/뮤지컬,미국,128분,12세이상관람가,데미안셔젤,"라이언고슬링,엠마스톤","황홀한 사랑, 순수한 희망, 격렬한 열정…이 곳에서 모든 감정이 폭발한다!꿈을 꾸는..."
뱅가드,5.3,/moviedb/main?movieId=131305,2020-12-30,코미디/액션/어드벤처,중국,108분,12세이상관람가,당계례,"성룡,양양,애륜",‘탕환팅’(성룡)이 이끄는 국제 민간 경호업체 뱅가드.새해 축제가 한창인 런던 한복...
도굴,7.2,/moviedb/main?movieId=131516,2020-11-04,범죄,한국,114분,12세이상관람가,박정배,"이제훈,조우진,신혜선,임원희",“고물인 줄 알았는데 보물이었다?!”땅 파서 장사하는 도굴꾼들이 온다!흙 맛만 봐도...
